In [1]:
%load_ext autoreload
%autoreload 1
%aimport ecg_get_data
%aimport Models
%aimport train_test_validat
from  ecg_get_data import *
import neurokit2
import matplotlib.pyplot as plt

import torch
import torch.utils.data as Data
import random


random_seed = 1
torch.manual_seed(random_seed)    # reproducible
torch.cuda.manual_seed_all(random_seed)
random.seed(random_seed)
np.random.seed(random_seed)

In [2]:
data_path =  '/workspace/data/OneDrive - mail.hfut.edu.cn/ECG/Interpretable_HTN/npy_ECG/' #路径
# data_path =   '/workspace/data/OneDrive - mail.hfut.edu.cn/ECG/Interpretable_HTN/small_set_for_testing'
lable_path = '/workspace/data/OneDrive - mail.hfut.edu.cn/ECG/Interpretable_HTN/label.npy'
model_path = '/workspace/data/OneDrive - mail.hfut.edu.cn/ECG/Interpretable_HTN/model/'

In [3]:
x = load_data(data_path,EcgChannles_num=12,EcgLength_num=5000)
y = load_label(lable_path)

100%|██████████| 1698/1698 [00:10<00:00, 160.90it/s]


In [4]:
x.shape

(1698, 12, 5000)

In [5]:
y.shape

(1698,)

In [6]:
train_dataset,valid_dataset = load_numpy_dataset_to_tensor_dataset(x,y,random_seed=random_seed,train_rate = 0.8)
#train_dataset = load_small_numpy_dataset_to_tensor_dataset(x,y,random_seed)
#valid_dataset = train_dataset

1698
               HTN  NHTN
valid_dataset: 149  191
train_dataset: 549  1209


In [7]:
BATCH_SIZE = 128
FOLDS = 1
EPOCHS = 50  
PATIENCE = 50
LR = 0.00001
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda:0


In [8]:
import Models 

NET = Models.CNN()
NET.to(DEVICE)

#定义优化器和损失函数
optimizer  = torch.optim.Adam(NET.parameters(), lr=LR)  
criterion = torch.nn.CrossEntropyLoss()   

In [9]:
from train_test_validat import *
from torch.optim.lr_scheduler import CosineAnnealingLR
import time

for fold in range(FOLDS):
    train_dataloader = Data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    valid_dataloader = Data.DataLoader(dataset=valid_dataset, batch_size=BATCH_SIZE, shuffle=True)
    #clr = CosineAnnealingLR(optimizer,T_max = 32) # 使用余弦退火算法改变学习率
    best_loss = 3
    for epoch in range(1,EPOCHS):
        time_all=0
        start_time = time.time()
        train_loss,train_acc = train_model(train_dataloader, NET, criterion, optimizer,DEVICE) # 训练模型
        #clr.step() # 学习率迭代
        time_all = time.time()-start_time
        valid_loss,valid_acc = test_model(train_dataloader,criterion,NET,DEVICE) # 测试模型
        print('- Epoch: %d - Train_loss: %.5f - Train_acc: %.5f - Val_loss: %.5f - Val_acc: %5f - T_Time: %.3f' %(epoch,train_loss,train_acc,valid_loss,valid_acc,time_all))
        print('当前学习率：%f' %optimizer.state_dict()['param_groups'][0]['lr'])
        if valid_loss < best_loss:
            best_loss = valid_loss
            print('Find better model in Epoch {0}, saving model.'.format(epoch))
            torch.save(NET.state_dict(),  model_path+'/best_model_' + str(fold) + '.pt')  # 保存最优模型
    print('Fold %d Training Finished' %(fold+1))
    torch.cuda.empty_cache()
print('Training Finished')

labels: tensor([0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0,
        1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0,
        0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0,
        0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
        0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0,
        1, 0, 1, 0, 1, 0, 0, 1], device='cuda:0')
pred: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
labels: tensor([1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0